---
# Models Perdormance Assesment 
---
- model perdormance is assesed based on model selection which is based on model features. The goal is to test weather the ensemble and applying a threshold will improve the model performance
- In the model selection below, one variable is isolated at a time (most of the time) , in order to asses its impact on the performance 
- Further analysis will combine well performing model features ... 
- In order to these results to be impactful, they need to persist across diferent years of backtesting  

In [1]:
import pandas as pd
import pickle
import random
import numpy as np
import os
import itertools
from joblib import Parallel, delayed , parallel_backend
from collections import defaultdict
import math
import torch.nn as nn
import json
import matplotlib.pyplot as plt
import pickle
from Equations_Run_Combo_V_2 import *
from Functions_Model_Processing import *
import pickle
from Functions_Analytic_1_Initial_GS_and_Dist_Analytics_HOD import *
from Functions_1_Initial_GS_and_Dist_Analytics_HOD import *


with open('/Users/cs/Desktop/LSTM_ETF_V3/short_dfs.pkl', 'rb') as f:
    loaded_dfs = pickle.load(f)

with open("/Users/cs/Desktop/LSTM_ETF_V3/lagged_cache.pkl", "rb") as f:
    lagged_cache = pickle.load(f)


#/home/charifslmn/




COMPUTE_DIST_RES =  True
COMPUTE_SINGLE_RES = False


use_one_realization_OUTER_2_VAR = False
number_realizations_OUTER_2_VAR = 15
number_realizations_outer_1_VAR = 15

use_one_realization_OUTER_2_VAR_SINGLE = True
number_realizations_OUTER_2_VAR_SINGLE = 15
number_realizations_outer_1_VAR_SINGLE = 15

print_ensemble_results = False
plot_distributions_notebook = False


In [2]:
#                                                       DATA SETS

file_name = "/Users/cs/Desktop/DATA_LSTM_ETF_V2/Vset_NOTsorted_90_models_GS_21_01_to_22_12_DIST_Discovery_10percentPOS_UCO.json" 

results_dist_disc = organize_dist_disc_dict(file_name)  



results_dist_disc_2 = []

for i in results_dist_disc:
    if (i['combo']["use_custom_loss_function_BCE_THRESH_AND_SEVERITY"] == True): # or i['combo']["use_custom_loss_function_BCE_THRESH"] == True):
        results_dist_disc_2.append(i)

len(results_dist_disc_2 )

results_dist_disc = results_dist_disc_2
    
file_name_Tset = "/Users/cs/Desktop/DATA_LSTM_ETF_V2/Tset_NOTsorted_90_models_GS_21_01_to_22_12_DIST_Discovery_10percentPOS_UCO.json"

results_dist_disc_Tset_same_seeds_organized = organize_dist_disc_Tset_dict(file_name_Tset, results_dist_disc=results_dist_disc)



In [3]:
len(results_dist_disc_Tset_same_seeds_organized) 

51

---
# Distibution of Preformances for Tested Params 

- plotting the distributions of the above results over multiple realization 
---

In [4]:
####                                   Model Selection (seeds chosen at random)

 - mean_precision_range , min_ratio_difference , max_FP_severe_ratio_FPs


Mp_H__param_map_1 = {   
               "mean_precision_range" : (60, 100),
               "max_zero_precision_seeds" : 70,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),

               
               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,
               "seed_precision_range" : (0, 100), "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None                                  
                 }

Mp_M__param_map_1 = {   
               "mean_precision_range" : (40, 75),
               "max_zero_precision_seeds" : 70,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),

               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,
               "seed_precision_range" : (0, 100), "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }

Mp_L__param_map_1 = {   
               "mean_precision_range" : (5, 40),
               "max_zero_precision_seeds" : 70,
               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),

               "seed_precision_range" : (0, 100), "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }

#############

Mp_ALL__Rfps_H__param_map_1 = {   
               "mean_precision_range" : (5, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : 0.4, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               
               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "seed_precision_range" : (0, 100), "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }


Mp_ALL__Rfps_L__param_map_1 = {   
               "mean_precision_range" : (5, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : 0.25, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               
               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "seed_precision_range" : (1, 40),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }


unDES_Mp_ALL__Rfps_H__param_map__param_map_1 = {   
               "mean_precision_range" : (5, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : 0.7, "min_FP_severe_ratio_FPs" : .5,         
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "seed_precision_range" : (0, 100),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }

### 
Mp_ALL__R_L__param_map_1 = {   
               "mean_precision_range" : (5, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),

               "min_ratio_difference" : 0.1, "max_ratio_difference" : .3,
               "seed_precision_range" : (0, 100),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }


Mp_ALL__R_H__param_map_1 = {  
               "mean_precision_range" : (5, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),

               "min_ratio_difference" : .15, "max_ratio_difference" : None ,
               "seed_precision_range" : (0, 100),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }

unDES_Mp_ALL__R_neg__param_map_1 = {   
               "mean_precision_range" : (5, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,         
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               "min_ratio_difference" : None, "max_ratio_difference" : 0,
               "seed_precision_range" : (0, 100), "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }

# ###################



Mp_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__param_map_1)
Mp_M__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_M__param_map_1)
Mp_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_L__param_map_1)


Mp_ALL__Rfps_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_ALL__Rfps_H__param_map_1)
Mp_ALL__Rfps_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_ALL__Rfps_L__param_map_1)
unDES_Mp_ALL__Rfps_H__param_map__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_ALL__Rfps_H__param_map__param_map_1)


Mp_ALL__R_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_ALL__R_L__param_map_1)
Mp_ALL__R_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_ALL__R_H__param_map_1)
unDES_Mp_ALL__R_neg__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_ALL__R_neg__param_map_1)

models_res_names_dict = { ## print model seleciton info , 
    "DES_Mp_H": Mp_H__param_map_models,
    "DES_Mp_M": Mp_M__param_map_models,
    "DES_Mp_L": Mp_L__param_map_models,

    "Mp_ALL__Rfps_H": Mp_ALL__Rfps_H__param_map_models,
    "Mp_ALL__Rfps_L": Mp_ALL__Rfps_L__param_map_models,
    "unDES_Mp_ALL__Rfps_H__param_map": unDES_Mp_ALL__Rfps_H__param_map__param_map_models,

    "Mp_ALL__R_L": Mp_ALL__R_L__param_map_models,
    "Mp_ALL__R_H": Mp_ALL__R_H__param_map_models,
    "unDES_Mp_ALL__R_neg": unDES_Mp_ALL__R_neg__param_map_models

}   

for k, selected_models in models_res_names_dict.items():
   print(f"Models Selected {k}: {len(selected_models)}")
   total_seeds = sum(len(model["selected_seeds"]) for model in selected_models)
   print(f"Total seeds selected {k}: {total_seeds}")
   print("-" * 5)



# #### collect raw V and T set data for the selected models
# Mp_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# Mp_M__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_M__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# Mp_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

# Mp_ALL__Rfps_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_ALL__Rfps_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# Mp_ALL__Rfps_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_ALL__Rfps_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# unDES_Mp_ALL__Rfps_H__param_map__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_ALL__Rfps_H__param_map__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

# Mp_ALL__R_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_ALL__R_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# Mp_ALL__R_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_ALL__R_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# unDES_Mp_ALL__R_neg__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_ALL__R_neg__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized) 
# #### collect raw V and T set data for the selected models

# model_based_selection_maps_raw_V_T_data = {
#     "Mp_H": Mp_H__data_V_T_rawData,
#     "Mp_M": Mp_M__data_V_T_rawData,
#     "Mp_L": Mp_L__data_V_T_rawData,

#     "Mp_ALL__Rfps_H": Mp_ALL__Rfps_H__data_V_T_rawData,
#     "Mp_ALL__Rfps_L": Mp_ALL__Rfps_L__data_V_T_rawData,
#     "unDES_Mp_ALL__Rfps_H__param_map": unDES_Mp_ALL__Rfps_H__param_map__data_V_T_rawData,

#     "Mp_ALL__R_L": Mp_ALL__R_L__data_V_T_rawData,
#     "Mp_ALL__R_H": Mp_ALL__R_H__data_V_T_rawData,
#     "unDES_Mp_ALL__R_neg": unDES_Mp_ALL__R_neg__data_V_T_rawData
# }

# import pickle
# with open('model_based_selection_maps_raw_V_T_data.pkl', 'wb') as f:
#     pickle.dump(model_based_selection_maps_raw_V_T_data, f)


Models Selected DES_Mp_H: 18
Total seeds selected DES_Mp_H: 692
-----
Models Selected DES_Mp_M: 46
Total seeds selected DES_Mp_M: 1765
-----
Models Selected DES_Mp_L: 4
Total seeds selected DES_Mp_L: 141
-----
Models Selected Mp_ALL__Rfps_H: 50
Total seeds selected Mp_ALL__Rfps_H: 1906
-----
Models Selected Mp_ALL__Rfps_L: 24
Total seeds selected Mp_ALL__Rfps_L: 250
-----
Models Selected unDES_Mp_ALL__Rfps_H__param_map: 0
Total seeds selected unDES_Mp_ALL__Rfps_H__param_map: 0
-----
Models Selected Mp_ALL__R_L: 10
Total seeds selected Mp_ALL__R_L: 400
-----
Models Selected Mp_ALL__R_H: 10
Total seeds selected Mp_ALL__R_H: 379
-----
Models Selected unDES_Mp_ALL__R_neg: 13
Total seeds selected unDES_Mp_ALL__R_neg: 502
-----


In [5]:
####                                   Model Selection (SEED filter only) ---- seed filter only to see if the dist discovery inc the prec 

Sp_H__param_map_2 = {   
               "mean_precision_range" : (5, 100),"max_zero_precision_seeds" : 70,"max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None, "min_ratio_difference" : None, "max_ratio_difference" : None,
               
               "seed_precision_range" : (75, 100) , 
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               "seed_min_TPminusFP_greaterEqual" : None , "max_seed_severe_FPs_high_bracket" : None , "min_seed_TPs_high_bracket" : None ,
                "min_seeds_per_model" : 1,"max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 

                 }

Sp_M__param_map_2 = {   
               "mean_precision_range" : (5, 100),"max_zero_precision_seeds" : 70, "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,"min_ratio_difference" : None, "max_ratio_difference" : None,
               
               "seed_precision_range" : (40, 75) , 
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               "seed_min_TPminusFP_greaterEqual" : None , "max_seed_severe_FPs_high_bracket" : None , 
               "min_seed_TPs_high_bracket" : None ,
                "min_seeds_per_model" : 1,"max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 
                 }

Sp_L__param_map_2 = {   
               "mean_precision_range" : (5, 100),"max_zero_precision_seeds" : 70,"max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None, "min_ratio_difference" : None, "max_ratio_difference" : None,

               "seed_precision_range" : (1, 40) , 
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               "seed_min_TPminusFP_greaterEqual" : None , "max_seed_severe_FPs_high_bracket" : None , 
               "min_seed_TPs_high_bracket" : None , 
                "min_seeds_per_model" : 1,"max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 
                 }

####

TH_Sp_ALL__Hb_0fp1tp__param_map_2 = {   
               "mean_precision_range" : (5, 100),"max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,         
               "min_ratio_difference" : None, "max_ratio_difference" : None,

               "seed_precision_range" : (5, 100) , 
               "seed_min_TPminusFP_greaterEqual" : None , 
               "max_seed_severe_FPs_high_bracket" : 0 , #**
               "min_seed_TPs_high_bracket" : 1 , #**
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               "min_seeds_per_model" : 1,
               "max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, 
               "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 

                 }

TH_Sp_ALL__Hb_1fp1tp__param_map_2 = {   
               "mean_precision_range" : (5, 100),"max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,         
               "min_ratio_difference" : None, "max_ratio_difference" : None,

               "seed_precision_range" : (5, 100) , 
               "seed_min_TPminusFP_greaterEqual" : None , 
               "max_seed_severe_FPs_high_bracket" : 1 , #**
               "min_seed_TPs_high_bracket" : 1 , #**
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
                "min_seeds_per_model" : 1,
               "max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, 
               "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 

                 }


TH_Sp_ALL__Hb_tp_minus_fp__param_map_2 = {   
               "mean_precision_range" : (5, 100),"max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,         
               "min_ratio_difference" : None, "max_ratio_difference" : None,

               "seed_precision_range" : (5, 100) , 
               "seed_min_TPminusFP_greaterEqual" : 3 , 
               "max_seed_severe_FPs_high_bracket" : None , 
               "min_seed_TPs_high_bracket" : None ,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
              "min_seeds_per_model" : 1,
               "max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, 
               "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 

                 }

####                                   Model Selection (SEED + MODEL Fileter)

TH__Sp_ALL__Hb_tp_minus_fp__param_map_2 = {   
               "mean_precision_range" : (10, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None,
               "min_FP_severe_ratio_FPs" : None,         
               "min_ratio_difference" : None,
               "max_ratio_difference" : None,

               "seed_precision_range" : (10, 100) , 
               "seed_min_TPminusFP_greaterEqual" : 2 , 
               "max_seed_severe_FPs_high_bracket" : None , 
               "min_seed_TPs_high_bracket" : None ,

               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),


               "min_seeds_per_model" : 3,
               "max_models_to_return" : 90,
               "max_FP_severe_ratio_FPs_andTPs" : None, 
               "min_FP_severe_ratio_FPs_andTPs" : None,

               "random_seed" : None

                 }


Sp_H__param_map_models = select_models_by_criteria(results_dist_disc, **Sp_H__param_map_2)
Sp_M__param_map_models = select_models_by_criteria(results_dist_disc, **Sp_M__param_map_2)
Sp_L__param_map_models = select_models_by_criteria(results_dist_disc, **Sp_L__param_map_2)

TH_Sp_ALL__Hb_0fp1tp__param_map_models = select_models_by_criteria(results_dist_disc, **TH_Sp_ALL__Hb_0fp1tp__param_map_2)
TH_Sp_ALL__Hb_1fp1tp__param_map_models = select_models_by_criteria(results_dist_disc, **TH_Sp_ALL__Hb_1fp1tp__param_map_2)
TH_Sp_ALL__Hb_tp_minus_fp__param_map_models = select_models_by_criteria(results_dist_disc, **TH_Sp_ALL__Hb_tp_minus_fp__param_map_2)


models_res_names_dict = {
    "Sp_H": Sp_H__param_map_models,
    "Sp_M": Sp_M__param_map_models,
    "Sp_L": Sp_L__param_map_models,


    "TH_Sp_ALL__Hb_0fp1tp": TH_Sp_ALL__Hb_0fp1tp__param_map_models,
    "TH_Sp_ALL__Hb_1fp1tp": TH_Sp_ALL__Hb_1fp1tp__param_map_models,
    "TH_Sp_ALL__Hb_tp_minus_fp": TH_Sp_ALL__Hb_tp_minus_fp__param_map_models,

}


for k , selected_models in models_res_names_dict.items():
   print(f"Models Selected {k}: {len(selected_models)}")
   total_seeds = sum(len(model["selected_seeds"]) for model in selected_models)
   print(f"Total seeds selected {k}: {total_seeds}")
   print("-" * 5)


# #### collect raw V and T set data for the selected models
# Sp_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Sp_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# Sp_M__data_V_T_rawData = collect_V_T_set_FULLraw_data(Sp_M__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# Sp_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Sp_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

# TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData = collect_V_T_set_FULLraw_data(TH_Sp_ALL__Hb_0fp1tp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# TH_Sp_ALL__Hb_1fp1tp__data_V_T_rawData = collect_V_T_set_FULLraw_data(TH_Sp_ALL__Hb_1fp1tp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# TH_Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData = collect_V_T_set_FULLraw_data(TH_Sp_ALL__Hb_tp_minus_fp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

# TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData = collect_V_T_set_FULLraw_data(TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# #### collect raw V and T set data for the selected models

# model_AND_seed_based_selection_maps_raw_V_T_data = {
#     "Sp_H": Sp_H__data_V_T_rawData,
#     "Sp_M": Sp_M__data_V_T_rawData,   
#     "Sp_L": Sp_L__data_V_T_rawData,

#     "TH_Sp_ALL__Hb_0fp1tp": TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData,
#     "TH_Sp_ALL__Hb_1fp1tp": TH_Sp_ALL__Hb_1fp1tp__data_V_T_rawData,
#     "TH_Sp_ALL__Hb_tp_minus_fp": TH_Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData,

#     "TH_Mp_H__Sp_ALL__Hb_tp_minus_fp": TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData
# }

# # with open('model_AND_seed_based_selection_maps_raw_V_T_data.pkl', 'wb') as f:
# #     pickle.dump(model_AND_seed_based_selection_maps_raw_V_T_data_Seed_and_Model, f)


Models Selected Sp_H: 41
Total seeds selected Sp_H: 222
-----
Models Selected Sp_M: 51
Total seeds selected Sp_M: 1408
-----
Models Selected Sp_L: 37
Total seeds selected Sp_L: 421
-----
Models Selected TH_Sp_ALL__Hb_0fp1tp: 33
Total seeds selected TH_Sp_ALL__Hb_0fp1tp: 490
-----
Models Selected TH_Sp_ALL__Hb_1fp1tp: 36
Total seeds selected TH_Sp_ALL__Hb_1fp1tp: 814
-----
Models Selected TH_Sp_ALL__Hb_tp_minus_fp: 29
Total seeds selected TH_Sp_ALL__Hb_tp_minus_fp: 228
-----


In [6]:
######################################                                                models selection code 

def run_results_realizations_1(use_one_realization_OUTER_2=True, number_realizations_OUTER_2=1, number_realizations_outer= number_realizations_outer_1_VAR):
    
    #### {model_key: {metric: [values across realizations]} , model_key2: {...} , ...}
    
    ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2 = []
    
    for i in range(number_realizations_OUTER_2):
        
        ALL_models_results_across_realizations = defaultdict(lambda: defaultdict(list))
        
        for j in range(number_realizations_outer):

            Mp_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__param_map_1)
            Mp_M__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_M__param_map_1)
            Mp_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_L__param_map_1)

            Mp_ALL__Rfps_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_ALL__Rfps_H__param_map_1)
            Mp_ALL__Rfps_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_ALL__Rfps_L__param_map_1)
            unDES_Mp_ALL__Rfps_H__param_map__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_ALL__Rfps_H__param_map__param_map_1)

            Mp_ALL__R_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_ALL__R_L__param_map_1)
            Mp_ALL__R_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_ALL__R_H__param_map_1)
            unDES_Mp_ALL__R_neg__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_ALL__R_neg__param_map_1)

            #### collect raw V and T set data for the selected models
            Mp_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_M__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_M__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

            Mp_ALL__Rfps_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_ALL__Rfps_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_ALL__Rfps_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_ALL__Rfps_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            unDES_Mp_ALL__Rfps_H__param_map__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_ALL__Rfps_H__param_map__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

            Mp_ALL__R_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_ALL__R_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_ALL__R_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_ALL__R_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            unDES_Mp_ALL__R_neg__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_ALL__R_neg__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized) 
            #### collect raw V and T set data for the selected models

            model_based_selection_maps_raw_V_T_data = {
                "Mp_H": Mp_H__data_V_T_rawData,
                "Mp_M": Mp_M__data_V_T_rawData,
                "Mp_L": Mp_L__data_V_T_rawData,

                "Mp_ALL__Rfps_H": Mp_ALL__Rfps_H__data_V_T_rawData,
                "Mp_ALL__Rfps_L": Mp_ALL__Rfps_L__data_V_T_rawData,
                "unDES_Mp_ALL__Rfps_H__param_map": unDES_Mp_ALL__Rfps_H__param_map__data_V_T_rawData,

                "Mp_ALL__R_L": Mp_ALL__R_L__data_V_T_rawData,
                "Mp_ALL__R_H": Mp_ALL__R_H__data_V_T_rawData,
                "unDES_Mp_ALL__R_neg": unDES_Mp_ALL__R_neg__data_V_T_rawData
            }

            models_data_V_T = {
                "Mp_H" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H"]["T_set_data"]} ,
                "Mp_M" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_M"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_M"]["T_set_data"]} ,
                "Mp_L" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_L"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_L"]["T_set_data"]} ,
                ###
                "Mp_ALL__Rfps_H" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_ALL__Rfps_H"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_ALL__Rfps_H"]["T_set_data"]} ,
                "Mp_ALL__Rfps_L" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_ALL__Rfps_L"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_ALL__Rfps_L"]["T_set_data"]} ,
                "unDES_Mp_ALL__Rfps_H__param_map" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_ALL__Rfps_H__param_map"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_ALL__Rfps_H__param_map"]["T_set_data"]} ,
                ###
                "Mp_ALL__R_L" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_ALL__R_L"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_ALL__R_L"]["T_set_data"]} ,
                "Mp_ALL__R_H" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_ALL__R_H"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_ALL__R_H"]["T_set_data"]} ,
                "unDES_Mp_ALL__R_neg" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_ALL__R_neg"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_ALL__R_neg"]["T_set_data"]}
            }

            model_keys = list(models_data_V_T.keys())

            for k in model_keys:
                V_set = models_data_V_T[k]["V_set_data"]
                T_set = models_data_V_T[k]["T_set_data"]
                out = process_and_RETURN_analytics_2_3_Model_Performance(V_set, T_set, do_print=False)
                
                ALL_models_results_across_realizations[k]["REG_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE"])        
                ALL_models_results_across_realizations[k]["REG_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["REG_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_regular_UNIQUE"])
                
                ALL_models_results_across_realizations[k]["THR_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE"])
                ALL_models_results_across_realizations[k]["THR_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["THR_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_threshold_UNIQUE"])

        if not use_one_realization_OUTER_2:
            ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2.append(copy.deepcopy(dict(ALL_models_results_across_realizations)))

        if use_one_realization_OUTER_2:
            return dict(ALL_models_results_across_realizations)
            

    with open('/Users/cs/Desktop/LSTM_ETF_V3/Model Performance Data UCO 21_01 - 22_12 Sev Loss/LIST_FOR_DIST_ALL_models_results_across_realization_1.pkl', 'wb') as f:
        pickle.dump(ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2, f)



In [7]:


######################################                                                seeds AND models selection code 


def run_results_realizations_2(use_one_realization_OUTER_2=True, number_realizations_OUTER_2=1, number_realizations_outer= number_realizations_outer_1_VAR):
    
    #### {model_key: {metric: [values across realizations]} , model_key2: {...} , ...}
    
    ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2 = []
    
    for i in range(number_realizations_OUTER_2):
        
        ALL_models_results_across_realizations = defaultdict(lambda: defaultdict(list))
        
        for j in range(number_realizations_outer):

            Sp_H__param_map_models = select_models_by_criteria(results_dist_disc, **Sp_H__param_map_2)
            Sp_M__param_map_models = select_models_by_criteria(results_dist_disc, **Sp_M__param_map_2)
            Sp_L__param_map_models = select_models_by_criteria(results_dist_disc, **Sp_L__param_map_2)

            TH_Sp_ALL__Hb_0fp1tp__param_map_models = select_models_by_criteria(results_dist_disc, **TH_Sp_ALL__Hb_0fp1tp__param_map_2)
            TH_Sp_ALL__Hb_1fp1tp__param_map_models = select_models_by_criteria(results_dist_disc, **TH_Sp_ALL__Hb_1fp1tp__param_map_2)
            TH_Sp_ALL__Hb_tp_minus_fp__param_map_models = select_models_by_criteria(results_dist_disc, **TH_Sp_ALL__Hb_tp_minus_fp__param_map_2)

            #### collect raw V and T set data for the selected models
            Sp_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Sp_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Sp_M__data_V_T_rawData = collect_V_T_set_FULLraw_data(Sp_M__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Sp_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Sp_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

            TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData = collect_V_T_set_FULLraw_data(TH_Sp_ALL__Hb_0fp1tp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            TH_Sp_ALL__Hb_1fp1tp__data_V_T_rawData = collect_V_T_set_FULLraw_data(TH_Sp_ALL__Hb_1fp1tp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            TH_Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData = collect_V_T_set_FULLraw_data(TH_Sp_ALL__Hb_tp_minus_fp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

            #### collect raw V and T set data for the selected models

            model_AND_seed_based_selection_maps_raw_V_T_data = {
                "Sp_H": Sp_H__data_V_T_rawData,
                "Sp_M": Sp_M__data_V_T_rawData,   
                "Sp_L": Sp_L__data_V_T_rawData,

                "TH_Sp_ALL__Hb_0fp1tp": TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData,
                "TH_Sp_ALL__Hb_1fp1tp": TH_Sp_ALL__Hb_1fp1tp__data_V_T_rawData,
                "TH_Sp_ALL__Hb_tp_minus_fp": TH_Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData,
            }

            models_AND_seeds_data_V_T = {
                "Sp_H" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Sp_H"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Sp_H"]["T_set_data"]} ,
                "Sp_M" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Sp_M"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Sp_M"]["T_set_data"]} ,
                "Sp_L" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Sp_L"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Sp_L"]["T_set_data"]} ,
                ###
                "TH_Sp_ALL__Hb_0fp1tp" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["TH_Sp_ALL__Hb_0fp1tp"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["TH_Sp_ALL__Hb_0fp1tp"]["T_set_data"]} ,
                "TH_Sp_ALL__Hb_1fp1tp" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["TH_Sp_ALL__Hb_1fp1tp"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["TH_Sp_ALL__Hb_1fp1tp"]["T_set_data"]} ,
                "TH_Sp_ALL__Hb_tp_minus_fp" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["TH_Sp_ALL__Hb_tp_minus_fp"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["TH_Sp_ALL__Hb_tp_minus_fp"]["T_set_data"]} ,
                ###
            }

            model_keys = list(models_AND_seeds_data_V_T.keys())

            for k in model_keys:
                V_set = models_AND_seeds_data_V_T[k]["V_set_data"]
                T_set = models_AND_seeds_data_V_T[k]["T_set_data"]
                out = process_and_RETURN_analytics_2_3_Model_Performance(V_set, T_set, do_print=False)
                
                ALL_models_results_across_realizations[k]["REG_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE"])
                ALL_models_results_across_realizations[k]["REG_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["REG_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_regular_UNIQUE"])

                ALL_models_results_across_realizations[k]["THR_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE"])
                ALL_models_results_across_realizations[k]["THR_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["THR_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_threshold_UNIQUE"])

        if not use_one_realization_OUTER_2:
            ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2.append(copy.deepcopy(dict(ALL_models_results_across_realizations)))

        if use_one_realization_OUTER_2:
            return dict(ALL_models_results_across_realizations)
            

    with open('/Users/cs/Desktop/LSTM_ETF_V3/Model Performance Data UCO 21_01 - 22_12 Sev Loss/LIST_FOR_DIST_ALL_models_results_across_realization_2.pkl', 'wb') as f:
        pickle.dump(ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2, f)



In [8]:
####                                   Model Selection (SEED filter only) ---- seed filter only to see if the dist discovery inc the prec 

Mp_H__Sp_H__param_map_3 = {   
                "mean_precision_range" : (60, 100),"max_zero_precision_seeds" : 70,"max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None, "min_ratio_difference" : None, "max_ratio_difference" : None,
                
                "seed_precision_range" : (75, 100) , 
                "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
                "seed_min_TPminusFP_greaterEqual" : None , "max_seed_severe_FPs_high_bracket" : None , "min_seed_TPs_high_bracket" : None ,
                "min_seeds_per_model" : 1,"max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 

                  }

Mp_H__Sp_M__param_map_3 = {   
                "mean_precision_range" : (60, 100),"max_zero_precision_seeds" : 70, "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,"min_ratio_difference" : None, "max_ratio_difference" : None,
                
                "seed_precision_range" : (40, 75) , 
                "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
                "seed_min_TPminusFP_greaterEqual" : None , "max_seed_severe_FPs_high_bracket" : None , 
                "min_seed_TPs_high_bracket" : None ,
                "min_seeds_per_model" : 1,"max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 
                  }

Mp_H__Sp_L__param_map_3 = {   
                "mean_precision_range" : (60, 100),"max_zero_precision_seeds" : 70,"max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None, "min_ratio_difference" : None, "max_ratio_difference" : None,

                "seed_precision_range" : (1, 40) , 
                "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
                "seed_min_TPminusFP_greaterEqual" : None , "max_seed_severe_FPs_high_bracket" : None , 
                "min_seed_TPs_high_bracket" : None , 
                "min_seeds_per_model" : 1,"max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 
                  }

####

Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_3 = {   
                "mean_precision_range" : (60, 100),"max_zero_precision_seeds" : 70,
                "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,         
                "min_ratio_difference" : None, "max_ratio_difference" : None,

                "seed_precision_range" : (5, 100) , 
                "seed_min_TPminusFP_greaterEqual" : None , 
                "max_seed_severe_FPs_high_bracket" : 0 , #**
                "min_seed_TPs_high_bracket" : 1 , #**
                "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
                "min_seeds_per_model" : 1,
                "max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, 
                "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 

                  }


####                                   Model Selection (SEED + MODEL Fileter)

TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_3 = {   
                "mean_precision_range" : (60, 100),
                "max_zero_precision_seeds" : 70,
                "max_FP_severe_ratio_FPs" : None,
                "min_FP_severe_ratio_FPs" : None,         
                "min_ratio_difference" : None, 
                "max_ratio_difference" : None,

                "seed_precision_range" : (10, 100) , 
                "seed_min_TPminusFP_greaterEqual" : 2 , 
                "max_seed_severe_FPs_high_bracket" : None , 
                "min_seed_TPs_high_bracket" : None ,

                "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),


                "min_seeds_per_model" : 1,
                "max_models_to_return" : 90,
                "max_FP_severe_ratio_FPs_andTPs" : None, 
                "min_FP_severe_ratio_FPs_andTPs" : None,

                "random_seed" : None

                  }


Mp_H__Sp_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_H__param_map_3)
Mp_H__Sp_M__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_M__param_map_3)
Mp_H__Sp_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_L__param_map_3)

Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_3)

TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models = select_models_by_criteria(results_dist_disc, **TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_3)


models_res_names_dict = {
    "Mp_H__Sp_H": Mp_H__Sp_H__param_map_models,
    "Mp_H__Sp_M": Mp_H__Sp_M__param_map_models,
    "Mp_H__Sp_L": Mp_H__Sp_L__param_map_models,

    "Mp_H__TH_Sp_ALL__Hb_0fp1tp": Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_models,

    "TH_Mp_H__Sp_ALL__Hb_tp_minus_fp": TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models
}


for k , selected_models in models_res_names_dict.items():
   print(f"Models Selected {k}: {len(selected_models)}")
   total_seeds = sum(len(model["selected_seeds"]) for model in selected_models)
   print(f"Total seeds selected {k}: {total_seeds}")
   print("-" * 5)


# #### collect raw V and T set data for the selected models
# Mp_H__Sp_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# Mp_H__Sp_M__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_M__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# Mp_H__Sp_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

# Mp_H__TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)


# Mp_H__TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData = collect_V_T_set_FULLraw_data(TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# #### collect raw V and T set data for the selected models

# V_2_MP_H__model_AND_seed_based_selection_maps_raw_V_T_data = {
#     "Sp_H": Mp_H__Sp_H__data_V_T_rawData,
#     "Sp_M": Mp_H__Sp_M__data_V_T_rawData,
#     "Sp_L": Mp_H__Sp_L__data_V_T_rawData,

#     "TH_Sp_ALL__Hb_0fp1tp": Mp_H__TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData,

#     "TH_Mp_H__Sp_ALL__Hb_tp_minus_fp": TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData
# }

# # with open('model_AND_seed_based_selection_maps_raw_V_T_data.pkl', 'wb') as f:
# #     pickle.dump(model_AND_seed_based_selection_maps_raw_V_T_data_Seed_and_Model, f)


Models Selected Mp_H__Sp_H: 14
Total seeds selected Mp_H__Sp_H: 146
-----
Models Selected Mp_H__Sp_M: 18
Total seeds selected Mp_H__Sp_M: 580
-----
Models Selected Mp_H__Sp_L: 4
Total seeds selected Mp_H__Sp_L: 17
-----
Models Selected Mp_H__TH_Sp_ALL__Hb_0fp1tp: 9
Total seeds selected Mp_H__TH_Sp_ALL__Hb_0fp1tp: 148
-----
Models Selected TH_Mp_H__Sp_ALL__Hb_tp_minus_fp: 7
Total seeds selected TH_Mp_H__Sp_ALL__Hb_tp_minus_fp: 129
-----


In [9]:
######################################                                                seeds AND models selection code 

def run_results_realizations_3(use_one_realization_OUTER_2=True, number_realizations_OUTER_2=1, number_realizations_outer= number_realizations_outer_1_VAR):
    
    #### {model_key: {metric: [values across realizations]} , model_key2: {...} , ...}
    
    ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2 = []
    
    for i in range(number_realizations_OUTER_2):
        
        ALL_models_results_across_realizations = defaultdict(lambda: defaultdict(list))
        
        for j in range(number_realizations_outer):

            Mp_H__Sp_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_H__param_map_3)
            Mp_H__Sp_M__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_M__param_map_3)
            Mp_H__Sp_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_L__param_map_3)

            Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_3)

            TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models = select_models_by_criteria(results_dist_disc, **TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_3)

            #### collect raw V and T set data for the selected models
            Mp_H__Sp_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_H__Sp_M__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_M__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_H__Sp_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

            Mp_H__TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

            TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData = collect_V_T_set_FULLraw_data(TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            #### collect raw V and T set data for the selected models

            model_AND_seed_based_selection_maps_raw_V_T_data = {
                "Mp_H__Sp_H": Mp_H__Sp_H__data_V_T_rawData,
                "Mp_H__Sp_M": Mp_H__Sp_M__data_V_T_rawData,
                "Mp_H__Sp_L": Mp_H__Sp_L__data_V_T_rawData,

                "Mp_H__TH_Sp_ALL__Hb_0fp1tp": Mp_H__TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData,

                "TH_Mp_H__Sp_ALL__Hb_tp_minus_fp": TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData
            }

            models_AND_seeds_data_V_T = {
                "Mp_H__Sp_H" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_H"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_H"]["T_set_data"]} ,
                "Mp_H__Sp_M" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_M"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_M"]["T_set_data"]} ,
                "Mp_H__Sp_L" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_L"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_L"]["T_set_data"]} ,
                ###
                "Mp_H__TH_Sp_ALL__Hb_0fp1tp" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__TH_Sp_ALL__Hb_0fp1tp"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__TH_Sp_ALL__Hb_0fp1tp"]["T_set_data"]} ,

                ###
                "TH_Mp_H__Sp_ALL__Hb_tp_minus_fp" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["TH_Mp_H__Sp_ALL__Hb_tp_minus_fp"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["TH_Mp_H__Sp_ALL__Hb_tp_minus_fp"]["T_set_data"]},
            }

            model_keys = list(models_AND_seeds_data_V_T.keys())

            for k in model_keys:
                V_set = models_AND_seeds_data_V_T[k]["V_set_data"]
                T_set = models_AND_seeds_data_V_T[k]["T_set_data"]
                out = process_and_RETURN_analytics_2_3_Model_Performance(V_set, T_set, do_print=False)
                
                ALL_models_results_across_realizations[k]["REG_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE"])
                ALL_models_results_across_realizations[k]["REG_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["REG_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_regular_UNIQUE"])

                ALL_models_results_across_realizations[k]["THR_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE"])
                ALL_models_results_across_realizations[k]["THR_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["THR_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_threshold_UNIQUE"])
        
        if not use_one_realization_OUTER_2:
            ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2.append(copy.deepcopy(dict(ALL_models_results_across_realizations)))

        if use_one_realization_OUTER_2:
            return dict(ALL_models_results_across_realizations)
            

    with open('/Users/cs/Desktop/LSTM_ETF_V3/Model Performance Data UCO 21_01 - 22_12 Sev Loss/LIST_FOR_DIST_ALL_models_results_across_realization_3.pkl', 'wb') as f:
        pickle.dump(ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2, f)



In [10]:
####                                   Model Selection (seeds chosen at random)

 - mean_precision_range , min_ratio_difference , max_FP_severe_ratio_FPs


#############

Mp_H__Rfps_H__param_map_4 = {   
               "mean_precision_range" : (40, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : 0.4, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               
               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "seed_precision_range" : (0, 100), "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }


Mp_H__Rfps_L__param_map_4 = {   
               "mean_precision_range" : (40, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : 0.2, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               
               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "seed_precision_range" : (1, 40),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }


unDES_Mp_H__Rfps_H__param_map_4 = {   
               "mean_precision_range" : (40, 100),

               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : 0.7, "min_FP_severe_ratio_FPs" : .5,         
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "seed_precision_range" : (0, 100),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }

### 
Mp_H__R_L__param_map_4 = {   
               "mean_precision_range" : (40, 100),

               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),

               "min_ratio_difference" : 0.1, "max_ratio_difference" : .3,
               "seed_precision_range" : (0, 100),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }


Mp_H__R_H__param_map_4 = {  
               "mean_precision_range" : (40, 100),

               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),

               "min_ratio_difference" : .2, "max_ratio_difference" : None ,
               "seed_precision_range" : (0, 100),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }

unDES_Mp_H__R_neg__param_map_4 = {   
               "mean_precision_range" : (40, 100),

               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,         
               "mean_recall_up_range" : (0, 100), "seed_recall_range" : (0, 100),
               "min_ratio_difference" : None, "max_ratio_difference" : 0,
               "seed_precision_range" : (0, 100), "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }

# ###################





Mp_H__Rfps_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Rfps_H__param_map_4)
Mp_H__Rfps_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Rfps_L__param_map_4)
unDES_Mp_H__Rfps_H__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_H__Rfps_H__param_map_4)


Mp_H__R_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__R_L__param_map_4)
Mp_H__R_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__R_H__param_map_4)
unDES_Mp_H__R_neg__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_H__R_neg__param_map_4)


models_res_names_dict = { ## print model seleciton info , 

    "Mp_H__Rfps_H": Mp_H__Rfps_H__param_map_models,
    "Mp_H__Rfps_L": Mp_H__Rfps_L__param_map_models,
    "Mp_H__Rfps_L": Mp_H__Rfps_L__param_map_models,
    "unDES_Mp_H__Rfps_H": unDES_Mp_H__Rfps_H__param_map_models,

    "Mp_H__R_L": Mp_H__R_L__param_map_models,
    "Mp_H__R_H": Mp_H__R_H__param_map_models,
    "unDES_Mp_H__R_neg": unDES_Mp_H__R_neg__param_map_models

}   

for k, selected_models in models_res_names_dict.items():
   print(f"Models Selected {k}: {len(selected_models)}")
   total_seeds = sum(len(model["selected_seeds"]) for model in selected_models)
   print(f"Total seeds selected {k}: {total_seeds}")
   print("-" * 5)



#### collect raw V and T set data for the selected models

Mp_H__Rfps_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Rfps_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
Mp_H__Rfps_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Rfps_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
unDES_Mp_H__Rfps_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_H__Rfps_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

Mp_H__R_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__R_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
Mp_H__R_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__R_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
unDES_Mp_H__R_neg__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_H__R_neg__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
#### collect raw V and T set data for the selected models

model_based_selection_maps_raw_V_T_data = {

    "Mp_H__Rfps_H": Mp_H__Rfps_H__data_V_T_rawData,
    "Mp_H__Rfps_L": Mp_H__Rfps_L__data_V_T_rawData,
    "unDES_Mp_H__Rfps_H": unDES_Mp_H__Rfps_H__data_V_T_rawData,

    "Mp_H__R_L": Mp_H__R_L__data_V_T_rawData,
    "Mp_H__R_H": Mp_H__R_H__data_V_T_rawData,
    "unDES_Mp_H__R_neg": unDES_Mp_H__R_neg__data_V_T_rawData
}

# import pickle
# with open('model_based_selection_maps_raw_V_T_data.pkl', 'wb') as f:
#     pickle.dump(model_based_selection_maps_raw_V_T_data, f)


Models Selected Mp_H__Rfps_H: 46
Total seeds selected Mp_H__Rfps_H: 1765
-----
Models Selected Mp_H__Rfps_L: 14
Total seeds selected Mp_H__Rfps_L: 108
-----
Models Selected unDES_Mp_H__Rfps_H: 0
Total seeds selected unDES_Mp_H__Rfps_H: 0
-----
Models Selected Mp_H__R_L: 10
Total seeds selected Mp_H__R_L: 400
-----
Models Selected Mp_H__R_H: 7
Total seeds selected Mp_H__R_H: 259
-----
Models Selected unDES_Mp_H__R_neg: 11
Total seeds selected unDES_Mp_H__R_neg: 436
-----


In [11]:
######################################                                                models selection code 

def run_results_realizations_4(use_one_realization_OUTER_2=True, number_realizations_OUTER_2=1, number_realizations_outer= number_realizations_outer_1_VAR):
    
    #### {model_key: {metric: [values across realizations]} , model_key2: {...} , ...}
    
    ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2 = []
    
    for i in range(number_realizations_OUTER_2):
        
        ALL_models_results_across_realizations = defaultdict(lambda: defaultdict(list))
        
        for j in range(number_realizations_outer):

            Mp_H__Rfps_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Rfps_H__param_map_4)
            Mp_H__Rfps_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Rfps_L__param_map_4)
            unDES_Mp_H__Rfps_H__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_H__Rfps_H__param_map_4)

            Mp_H__R_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__R_L__param_map_4)
            Mp_H__R_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__R_H__param_map_4)
            unDES_Mp_H__R_neg__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_H__R_neg__param_map_4)

            #### collect raw V and T set data for the selected models
            Mp_H__Rfps_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Rfps_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_H__Rfps_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Rfps_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            unDES_Mp_H__Rfps_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_H__Rfps_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

            Mp_H__R_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__R_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_H__R_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__R_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            unDES_Mp_H__R_neg__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_H__R_neg__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized) 
            #### collect raw V and T set data for the selected models

            model_based_selection_maps_raw_V_T_data = {
                "Mp_H__Rfps_H": Mp_H__Rfps_H__data_V_T_rawData,
                "Mp_H__Rfps_L": Mp_H__Rfps_L__data_V_T_rawData,
                "unDES_Mp_H__Rfps_H": unDES_Mp_H__Rfps_H__data_V_T_rawData,

                "Mp_H__R_L": Mp_H__R_L__data_V_T_rawData,
                "Mp_H__R_H": Mp_H__R_H__data_V_T_rawData,
                "unDES_Mp_H__R_neg": unDES_Mp_H__R_neg__data_V_T_rawData
            }

            models_data_V_T = {
                "Mp_H__Rfps_H" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__Rfps_H"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__Rfps_H"]["T_set_data"]} ,
                "Mp_H__Rfps_L" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__Rfps_L"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__Rfps_L"]["T_set_data"]} ,
                "unDES_Mp_H__Rfps_H" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_H__Rfps_H"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_H__Rfps_H"]["T_set_data"]} ,
                ###
                "Mp_H__R_L" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__R_L"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__R_L"]["T_set_data"]} ,
                "Mp_H__R_H" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__R_H"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__R_H"]["T_set_data"]} ,
                "unDES_Mp_H__R_neg" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_H__R_neg"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_H__R_neg"]["T_set_data"]}
            }

            model_keys = list(models_data_V_T.keys())

            for k in model_keys:
                V_set = models_data_V_T[k]["V_set_data"]
                T_set = models_data_V_T[k]["T_set_data"]
                out = process_and_RETURN_analytics_2_3_Model_Performance(V_set, T_set, do_print=False)
                
                ALL_models_results_across_realizations[k]["REG_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE"])        
                ALL_models_results_across_realizations[k]["REG_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["REG_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_regular_UNIQUE"])
                
                ALL_models_results_across_realizations[k]["THR_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE"])
                ALL_models_results_across_realizations[k]["THR_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["THR_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_threshold_UNIQUE"])

        if not use_one_realization_OUTER_2:
            ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2.append(copy.deepcopy(dict(ALL_models_results_across_realizations)))

        if use_one_realization_OUTER_2:
            return dict(ALL_models_results_across_realizations)
            

    with open('/Users/cs/Desktop/LSTM_ETF_V3/Model Performance Data UCO 21_01 - 22_12 Sev Loss/LIST_FOR_DIST_ALL_models_results_across_realization_4.pkl', 'wb') as f:
        pickle.dump(ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2, f)



In [12]:
#*#*#*#*#   NOTE        LOW RECALL TEST 


####                                   Model Selection (seeds chosen at random)

 - mean_precision_range , min_ratio_difference , max_FP_severe_ratio_FPs


#############

Mp_H__Rfps_H__param_map_5 = {   
               "mean_precision_range" : (40, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : 0.4, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 30), "seed_recall_range" : (0, 30),
               
               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "seed_precision_range" : (0, 100), "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }


Mp_H__Rfps_L__param_map_5 = {   
               "mean_precision_range" : (40, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : 0.2, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 30), "seed_recall_range" : (0, 30),


               
               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "seed_precision_range" : (1, 40),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }


unDES_Mp_H__Rfps_H__param_map_5 = {   
               "mean_precision_range" : (40, 100),

               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : 0.7, "min_FP_severe_ratio_FPs" : .5,         
               "mean_recall_up_range" : (0, 30), "seed_recall_range" : (0, 30),


               "min_ratio_difference" : None, "max_ratio_difference" : None,
               "seed_precision_range" : (0, 100),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }

### 
Mp_H__R_L__param_map_5 = {   
               "mean_precision_range" : (40, 100),

               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 30), "seed_recall_range" : (0, 30),



               "min_ratio_difference" : 0.1, "max_ratio_difference" : .3,
               "seed_precision_range" : (0, 100),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }


Mp_H__R_H__param_map_5 = {  
               "mean_precision_range" : (40, 100),

               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,
               "mean_recall_up_range" : (0, 30), "seed_recall_range" : (0, 40),



               "min_ratio_difference" : .3, "max_ratio_difference" : None ,
               "seed_precision_range" : (0, 100),  "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }

unDES_Mp_H__R_neg__param_map_5 = {   
               "mean_precision_range" : (50, 100),

               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,         
               "mean_recall_up_range" : (0, 30), "seed_recall_range" : (0, 30),


               "min_ratio_difference" : None, "max_ratio_difference" : 0,
               "seed_precision_range" : (0, 100), "min_seeds_per_model" : 1,"max_models_to_return" : 90, "seed_min_TPminusFP_greaterEqual" : None,"max_FP_severe_ratio_FPs_andTPs" : None, 
               "max_seed_severe_FPs_high_bracket" : None,"min_seed_TPs_high_bracket" : None, "random_seed" : None , "min_FP_severe_ratio_FPs_andTPs" : None
                 }

# ###################





Mp_H__Rfps_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Rfps_H__param_map_5)
Mp_H__Rfps_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Rfps_L__param_map_5)
unDES_Mp_H__Rfps_H__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_H__Rfps_H__param_map_5)


Mp_H__R_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__R_L__param_map_5)
Mp_H__R_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__R_H__param_map_5)
unDES_Mp_H__R_neg__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_H__R_neg__param_map_5)


models_res_names_dict = { ## print model seleciton info , 

    "Mp_H__Rfps_H": Mp_H__Rfps_H__param_map_models,
    "Mp_H__Rfps_L": Mp_H__Rfps_L__param_map_models,
    "Mp_H__Rfps_L": Mp_H__Rfps_L__param_map_models,
    "unDES_Mp_H__Rfps_H": unDES_Mp_H__Rfps_H__param_map_models,

    "Mp_H__R_L": Mp_H__R_L__param_map_models,
    "Mp_H__R_H": Mp_H__R_H__param_map_models,
    "unDES_Mp_H__R_neg": unDES_Mp_H__R_neg__param_map_models

}   

for k, selected_models in models_res_names_dict.items():
   print(f"Models Selected {k}: {len(selected_models)}")
   total_seeds = sum(len(model["selected_seeds"]) for model in selected_models)
   print(f"Total seeds selected {k}: {total_seeds}")
   print("-" * 5)



#### collect raw V and T set data for the selected models

Mp_H__Rfps_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Rfps_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
Mp_H__Rfps_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Rfps_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
unDES_Mp_H__Rfps_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_H__Rfps_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

Mp_H__R_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__R_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
Mp_H__R_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__R_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
unDES_Mp_H__R_neg__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_H__R_neg__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
#### collect raw V and T set data for the selected models

model_based_selection_maps_raw_V_T_data = {

    "Mp_H__Rfps_H": Mp_H__Rfps_H__data_V_T_rawData,
    "Mp_H__Rfps_L": Mp_H__Rfps_L__data_V_T_rawData,
    "unDES_Mp_H__Rfps_H": unDES_Mp_H__Rfps_H__data_V_T_rawData,

    "Mp_H__R_L": Mp_H__R_L__data_V_T_rawData,
    "Mp_H__R_H": Mp_H__R_H__data_V_T_rawData,
    "unDES_Mp_H__R_neg": unDES_Mp_H__R_neg__data_V_T_rawData
}

# import pickle
# with open('model_based_selection_maps_raw_V_T_data.pkl', 'wb') as f:
#     pickle.dump(model_based_selection_maps_raw_V_T_data, f)


Models Selected Mp_H__Rfps_H: 16
Total seeds selected Mp_H__Rfps_H: 372
-----
Models Selected Mp_H__Rfps_L: 5
Total seeds selected Mp_H__Rfps_L: 55
-----
Models Selected unDES_Mp_H__Rfps_H: 0
Total seeds selected unDES_Mp_H__Rfps_H: 0
-----
Models Selected Mp_H__R_L: 4
Total seeds selected Mp_H__R_L: 104
-----
Models Selected Mp_H__R_H: 1
Total seeds selected Mp_H__R_H: 19
-----
Models Selected unDES_Mp_H__R_neg: 2
Total seeds selected unDES_Mp_H__R_neg: 47
-----


In [13]:
######################################                                                models selection code 

def run_results_realizations_5(use_one_realization_OUTER_2=True, number_realizations_OUTER_2=1, number_realizations_outer= number_realizations_outer_1_VAR):
    
    #### {model_key: {metric: [values across realizations]} , model_key2: {...} , ...}
    
    ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2 = []
    
    for i in range(number_realizations_OUTER_2):
        
        ALL_models_results_across_realizations = defaultdict(lambda: defaultdict(list))
        
        for j in range(number_realizations_outer):

            Mp_H__Rfps_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Rfps_H__param_map_5)
            Mp_H__Rfps_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Rfps_L__param_map_5)
            unDES_Mp_H__Rfps_H__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_H__Rfps_H__param_map_5)

            Mp_H__R_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__R_L__param_map_5)
            Mp_H__R_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__R_H__param_map_5)
            unDES_Mp_H__R_neg__param_map_models = select_models_by_criteria(results_dist_disc, **unDES_Mp_H__R_neg__param_map_5)

            #### collect raw V and T set data for the selected models
            Mp_H__Rfps_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Rfps_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_H__Rfps_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Rfps_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            unDES_Mp_H__Rfps_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_H__Rfps_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

            Mp_H__R_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__R_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_H__R_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__R_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            unDES_Mp_H__R_neg__data_V_T_rawData = collect_V_T_set_FULLraw_data(unDES_Mp_H__R_neg__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized) 
            #### collect raw V and T set data for the selected models

            model_based_selection_maps_raw_V_T_data = {
                "Mp_H__Rfps_H": Mp_H__Rfps_H__data_V_T_rawData,
                "Mp_H__Rfps_L": Mp_H__Rfps_L__data_V_T_rawData,
                "unDES_Mp_H__Rfps_H": unDES_Mp_H__Rfps_H__data_V_T_rawData,

                "Mp_H__R_L": Mp_H__R_L__data_V_T_rawData,
                "Mp_H__R_H": Mp_H__R_H__data_V_T_rawData,
                "unDES_Mp_H__R_neg": unDES_Mp_H__R_neg__data_V_T_rawData
            }

            models_data_V_T = {
                "Sr_L__Mr__L__Mp_H__Rfps_H" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__Rfps_H"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__Rfps_H"]["T_set_data"]} ,
                "Sr_L__Mr__L__Mp_H__Rfps_L" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__Rfps_L"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__Rfps_L"]["T_set_data"]} ,
                "Sr_L__Mr__L__unDES_Mp_H__Rfps_H" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_H__Rfps_H"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_H__Rfps_H"]["T_set_data"]} ,
                ###
                "Sr_L__Mr__L__Mp_H__R_L" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__R_L"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__R_L"]["T_set_data"]} ,
                "Sr_L__Mr__L__Mp_H__R_H" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__R_H"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["Mp_H__R_H"]["T_set_data"]} ,
                "Sr_L__Mr__L__unDES_Mp_H__R_neg" : {"V_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_H__R_neg"]["V_set_data"] , "T_set_data" : model_based_selection_maps_raw_V_T_data["unDES_Mp_H__R_neg"]["T_set_data"]}
            }

            model_keys = list(models_data_V_T.keys())

            for k in model_keys:
                V_set = models_data_V_T[k]["V_set_data"]
                T_set = models_data_V_T[k]["T_set_data"]
                out = process_and_RETURN_analytics_2_3_Model_Performance(V_set, T_set, do_print=False)
                
                ALL_models_results_across_realizations[k]["REG_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE"])        
                ALL_models_results_across_realizations[k]["REG_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["REG_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_regular_UNIQUE"])
                
                ALL_models_results_across_realizations[k]["THR_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE"])
                ALL_models_results_across_realizations[k]["THR_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["THR_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_threshold_UNIQUE"])

        if not use_one_realization_OUTER_2:
            ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2.append(copy.deepcopy(dict(ALL_models_results_across_realizations)))

        if use_one_realization_OUTER_2:
            return dict(ALL_models_results_across_realizations)
            
    with open('/Users/cs/Desktop/LSTM_ETF_V3/Model Performance Data UCO 21_01 - 22_12 Sev Loss/LIST_FOR_DIST_ALL_models_results_across_realization_5.pkl', 'wb') as f:
        pickle.dump(ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2, f)


In [14]:
#*#*#*#*#    NOTE      LOW RECALL TEST 
  

####                                   Model Selection (SEED filter only) ---- seed filter only to see if the dist discovery inc the prec 

Mp_H__Sp_H__param_map_6 = {   
               "mean_precision_range" : (60, 100),"max_zero_precision_seeds" : 70,"max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None, "min_ratio_difference" : None, "max_ratio_difference" : None,
               
               "seed_precision_range" : (65, 100) , 
               "mean_recall_up_range" : (0, 40), "seed_recall_range" : (0, 30),
               "seed_min_TPminusFP_greaterEqual" : None , "max_seed_severe_FPs_high_bracket" : None , "min_seed_TPs_high_bracket" : None ,
                "min_seeds_per_model" : 1,"max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 

                 }

Mp_H__Sp_M__param_map_6 = {   
               "mean_precision_range" : (60, 100),"max_zero_precision_seeds" : 70, "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,"min_ratio_difference" : None, "max_ratio_difference" : None,
               
               "seed_precision_range" : (40, 75) , 
               "mean_recall_up_range" : (0, 40), "seed_recall_range" : (0, 30),



               "seed_min_TPminusFP_greaterEqual" : None , "max_seed_severe_FPs_high_bracket" : None , 
               "min_seed_TPs_high_bracket" : None ,
                "min_seeds_per_model" : 1,"max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 
                 }

Mp_H__Sp_L__param_map_6 = {   
               "mean_precision_range" : (60, 100),"max_zero_precision_seeds" : 70,"max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None, "min_ratio_difference" : None, "max_ratio_difference" : None,

               "seed_precision_range" : (1, 50) , 
               "mean_recall_up_range" : (0, 40), "seed_recall_range" : (0, 30),



               "seed_min_TPminusFP_greaterEqual" : None , "max_seed_severe_FPs_high_bracket" : None , 
               "min_seed_TPs_high_bracket" : None , 
                "min_seeds_per_model" : 1,"max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 
                 }

####

Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_6 = {   
               "mean_precision_range" : (60, 100),"max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None, "min_FP_severe_ratio_FPs" : None,         
               "min_ratio_difference" : None, "max_ratio_difference" : None,

               "seed_precision_range" : (5, 100) , 
               "seed_min_TPminusFP_greaterEqual" : None , 
               "max_seed_severe_FPs_high_bracket" : 0 , #**
               "min_seed_TPs_high_bracket" : 1 , #**
               "mean_recall_up_range" : (0, 40), "seed_recall_range" : (0, 30),


                "min_seeds_per_model" : 1,
               "max_models_to_return" : 90, "max_FP_severe_ratio_FPs_andTPs" : None, 
               "min_FP_severe_ratio_FPs_andTPs" : None ,"random_seed" : None 

                 }


####                                   Model Selection (SEED + MODEL Fileter)

TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_6 = {   
               "mean_precision_range" : (55, 100),
               "max_zero_precision_seeds" : 70,
               "max_FP_severe_ratio_FPs" : None,
               "min_FP_severe_ratio_FPs" : None,         
               "min_ratio_difference" : None, 
               "max_ratio_difference" : None,

               "seed_precision_range" : (5, 100) , 
               "seed_min_TPminusFP_greaterEqual" : 2 ,  ## not enough models if this number is greater than 2 
               "max_seed_severe_FPs_high_bracket" : None , 
               "min_seed_TPs_high_bracket" : None,

               "mean_recall_up_range" : (0, 40), "seed_recall_range" : (0, 30),




               "min_seeds_per_model" : 1,
               "max_models_to_return" : 90,
               "max_FP_severe_ratio_FPs_andTPs" : None, 
               "min_FP_severe_ratio_FPs_andTPs" : None,

               "random_seed" : None

                 }


Mp_H__Sp_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_H__param_map_6)
Mp_H__Sp_M__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_M__param_map_6)
Mp_H__Sp_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_L__param_map_6)

Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_6)

TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models = select_models_by_criteria(results_dist_disc, **TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_6)


models_res_names_dict = {
    "Mp_H__Sp_H": Mp_H__Sp_H__param_map_models,
    "Mp_H__Sp_M": Mp_H__Sp_M__param_map_models,
    "Mp_H__Sp_L": Mp_H__Sp_L__param_map_models,


    "Mp_H__TH_Sp_ALL__Hb_0fp1tp": Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_models,

    "TH_Mp_H__Sp_ALL__Hb_tp_minus_fp": TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models
}


for k , selected_models in models_res_names_dict.items():
   print(f"Models Selected {k}: {len(selected_models)}")
   total_seeds = sum(len(model["selected_seeds"]) for model in selected_models)
   print(f"Total seeds selected {k}: {total_seeds}")
   print("-" * 5)


# #### collect raw V and T set data for the selected models
# Mp_H__Sp_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# Mp_H__Sp_M__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_M__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# Mp_H__Sp_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

# Mp_H__TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)


# TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData = collect_V_T_set_FULLraw_data(TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
# #### collect raw V and T set data for the selected models

# V_2_MP_H__model_AND_seed_based_selection_maps_raw_V_T_data = {
#     "Sp_H": Mp_H__Sp_H__data_V_T_rawData,
#     "Sp_M": Mp_H__Sp_M__data_V_T_rawData,
#     "Sp_L": Mp_H__Sp_L__data_V_T_rawData,

#     "TH_Sp_ALL__Hb_0fp1tp": Mp_H__TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData,

#     "TH_Mp_H__Sp_ALL__Hb_tp_minus_fp": TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData
# }

# # with open('model_AND_seed_based_selection_maps_raw_V_T_data.pkl', 'wb') as f:
# #     pickle.dump(model_AND_seed_based_selection_maps_raw_V_T_data_Seed_and_Model, f)


Models Selected Mp_H__Sp_H: 5
Total seeds selected Mp_H__Sp_H: 31
-----
Models Selected Mp_H__Sp_M: 4
Total seeds selected Mp_H__Sp_M: 36
-----
Models Selected Mp_H__Sp_L: 5
Total seeds selected Mp_H__Sp_L: 32
-----
Models Selected Mp_H__TH_Sp_ALL__Hb_0fp1tp: 2
Total seeds selected Mp_H__TH_Sp_ALL__Hb_0fp1tp: 11
-----
Models Selected TH_Mp_H__Sp_ALL__Hb_tp_minus_fp: 3
Total seeds selected TH_Mp_H__Sp_ALL__Hb_tp_minus_fp: 10
-----


In [15]:
######################################                                                seeds AND models selection code 

def run_results_realizations_6(use_one_realization_OUTER_2=True, number_realizations_OUTER_2=1, number_realizations_outer= number_realizations_outer_1_VAR):
    
    #### {model_key: {metric: [values across realizations]} , model_key2: {...} , ...}
    
    ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2 = []
    
    for i in range(number_realizations_OUTER_2):
        
        ALL_models_results_across_realizations = defaultdict(lambda: defaultdict(list))
        
        for j in range(number_realizations_outer):

            Mp_H__Sp_H__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_H__param_map_6)
            Mp_H__Sp_M__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_M__param_map_6)
            Mp_H__Sp_L__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__Sp_L__param_map_6)

            Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_models = select_models_by_criteria(results_dist_disc, **Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_6)

            TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models = select_models_by_criteria(results_dist_disc, **TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_6)


            # models_res_names_dict = {
            #     "Sp_H": Sp_H__param_map_models,
            #     "Sp_M": Sp_M__param_map_models,
            #     "Sp_L": Sp_L__param_map_models,


            #     "TH_Sp_ALL__Hb_0fp1tp": TH_Sp_ALL__Hb_0fp1tp__param_map_models,
            #     "TH_Sp_ALL__Hb_1fp1tp": TH_Sp_ALL__Hb_1fp1tp__param_map_models,
            #     "TH_Sp_ALL__Hb_tp_minus_fp": TH_Sp_ALL__Hb_tp_minus_fp__param_map_models,

            #     "TH_Mp_H__Sp_ALL__Hb_tp_minus_fp": TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models
            # }


            # for k , selected_models in models_res_names_dict.items():
            #    print(f"Models Selected {k}: {len(selected_models)}")
            #    total_seeds = sum(len(model["selected_seeds"]) for model in selected_models)
            #    print(f"Total seeds selected {k}: {total_seeds}")
            #    print("-" * 5)


            #### collect raw V and T set data for the selected models
            Mp_H__Sp_H__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_H__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_H__Sp_M__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_M__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            Mp_H__Sp_L__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__Sp_L__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)

            Mp_H__TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData = collect_V_T_set_FULLraw_data(Mp_H__TH_Sp_ALL__Hb_0fp1tp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)


            TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData = collect_V_T_set_FULLraw_data(TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__param_map_models, results_dist_disc, results_dist_disc_Tset_same_seeds_organized)
            #### collect raw V and T set data for the selected models

            model_AND_seed_based_selection_maps_raw_V_T_data = {
                "Mp_H__Sp_H": Mp_H__Sp_H__data_V_T_rawData,
                "Mp_H__Sp_M": Mp_H__Sp_M__data_V_T_rawData,
                "Mp_H__Sp_L": Mp_H__Sp_L__data_V_T_rawData,

                "Mp_H__TH_Sp_ALL__Hb_0fp1tp": Mp_H__TH_Sp_ALL__Hb_0fp1tp__data_V_T_rawData,

                "TH_Mp_H__Sp_ALL__Hb_tp_minus_fp": TH_Mp_H__Sp_ALL__Hb_tp_minus_fp__data_V_T_rawData
            }

            # with open('model_AND_seed_based_selection_maps_raw_V_T_data.pkl', 'wb') as f:
            #     pickle.dump(model_AND_seed_based_selection_maps_raw_V_T_data_Seed_and_Model, f)


            # dict_keys(['Sp_H', 'Sp_M', 'Sp_L', 'TH_Sp_ALL__Hb_0fp1tp', 'TH_Sp_ALL__Hb_1fp1tp', 
            #       'TH_Sp_ALL__Hb_tp_minus_fp', 'TH_Mp_H__Sp_ALL__Hb_tp_minus_fp'])
            models_AND_seeds_data_V_T = {
                "Sr_L__Mr__L__Mp_H__Sp_H" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_H"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_H"]["T_set_data"]} ,
                "Sr_L__Mr__L__Mp_H__Sp_M" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_M"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_M"]["T_set_data"]} ,
                "Sr_L__Mr__L__Mp_H__Sp_L" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_L"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__Sp_L"]["T_set_data"]} ,
                ###
                "Sr_L__Mr__L__Mp_H__TH_Sp_ALL__Hb_0fp1tp" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__TH_Sp_ALL__Hb_0fp1tp"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["Mp_H__TH_Sp_ALL__Hb_0fp1tp"]["T_set_data"]} ,

                ###
                "Sr_L__Mr__L__TH_Mp_H__Sp_ALL__Hb_tp_minus_fp" : {"V_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["TH_Mp_H__Sp_ALL__Hb_tp_minus_fp"]["V_set_data"] , "T_set_data" : model_AND_seed_based_selection_maps_raw_V_T_data["TH_Mp_H__Sp_ALL__Hb_tp_minus_fp"]["T_set_data"]},
            }

            model_keys = list(models_AND_seeds_data_V_T.keys())

            for k in model_keys:
                V_set = models_AND_seeds_data_V_T[k]["V_set_data"]
                T_set = models_AND_seeds_data_V_T[k]["T_set_data"]
                out = process_and_RETURN_analytics_2_3_Model_Performance(V_set, T_set, do_print=False)
                
                ALL_models_results_across_realizations[k]["REG_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE"])
                ALL_models_results_across_realizations[k]["REG_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_regular_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["REG_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_regular_UNIQUE"])

                
                
                ALL_models_results_across_realizations[k]["THR_UNIQUE_ALL"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE"])
                ALL_models_results_across_realizations[k]["THR_UNIQUE_SUM"].append(out["all_realizations_unique_actuals_ups_threshold_UNIQUE_SUM"])
                ALL_models_results_across_realizations[k]["THR_NUM_PREDS_UNIQUE"].append(out["LEN_all_realizations_unique_actuals_ups_threshold_UNIQUE"])

        if not use_one_realization_OUTER_2:
            ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2.append(copy.deepcopy(dict(ALL_models_results_across_realizations)))

        if use_one_realization_OUTER_2:
            return dict(ALL_models_results_across_realizations)
            

    with open('/Users/cs/Desktop/LSTM_ETF_V3/Model Performance Data UCO 21_01 - 22_12 Sev Loss/LIST_FOR_DIST_ALL_models_results_across_realization_6.pkl', 'wb') as f:
        pickle.dump(ALL_models_results_across_realizations_LIST_ALL_REALIZATIONS_OUTER_2, f)



In [17]:
from joblib import Parallel, delayed, parallel_backend

with parallel_backend('loky', n_jobs=-1):
    Parallel()(

    [ delayed(run_results_realizations_1)(use_one_realization_OUTER_2 = use_one_realization_OUTER_2_VAR , number_realizations_OUTER_2 = number_realizations_OUTER_2_VAR ,number_realizations_outer = number_realizations_outer_1_VAR) ,
    delayed(run_results_realizations_2)(use_one_realization_OUTER_2 = use_one_realization_OUTER_2_VAR , number_realizations_OUTER_2 = number_realizations_OUTER_2_VAR ,number_realizations_outer = number_realizations_outer_1_VAR) ,
    delayed(run_results_realizations_3)(use_one_realization_OUTER_2 = use_one_realization_OUTER_2_VAR , number_realizations_OUTER_2 = number_realizations_OUTER_2_VAR ,number_realizations_outer = number_realizations_outer_1_VAR), 
    delayed(run_results_realizations_4)(use_one_realization_OUTER_2 = use_one_realization_OUTER_2_VAR , number_realizations_OUTER_2 = number_realizations_OUTER_2_VAR ,number_realizations_outer = number_realizations_outer_1_VAR), 
    delayed(run_results_realizations_5)(use_one_realization_OUTER_2 = use_one_realization_OUTER_2_VAR , number_realizations_OUTER_2 = number_realizations_OUTER_2_VAR ,number_realizations_outer = number_realizations_outer_1_VAR), 
    delayed(run_results_realizations_6)(use_one_realization_OUTER_2 = use_one_realization_OUTER_2_VAR , number_realizations_OUTER_2 = number_realizations_OUTER_2_VAR ,number_realizations_outer = number_realizations_outer_1_VAR) ]


    )




[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[skip] pair_2_neg1_1: no items or missing 'parameters' for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[skip] pair_2_neg1_1: no items or missing 'parameters' for 'T'
[skip] pair_2_neg1_1: no items or missing 'parameters' for 'T'
[skip] pair_2_neg1_1: no items or missing 'parameters' for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles of size 2 for 'T'
[use] pair_2_neg1_1: 1 ensembles o

KeyboardInterrupt: 